In [132]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import matplotlib.animation as anm
import matplotlib.patches as patches
import math
import numpy as np

In [133]:
class World:
    def __init__(self,time_span,time_interval,debug=False):
        self.time_span=time_span
        self.time_interval=time_interval
        self.debug=debug
        self.objects=[]
    
    def append(self,obj):
        print("[debug] World: append")
        self.objects.append(obj)
    
    def draw(self):
        print("[debug] World: draw")
        fig=plt.figure(figsize=(3,3))
        ax=fig.add_subplot(111)
        ax.set_aspect('equal')
        ax.set_xlim(-5,5)
        ax.set_ylim(-5,5)
        ax.set_xlabel("X",fontsize=10)
        ax.set_ylabel("Y",fontsize=10)
        
        elems=[]
        
        if self.debug:
            for frame in range(3): self.one_step(frame,ax,elems)
        else:
            self.ani=anm.FuncAnimation(fig=fig,func=self.one_step,frames=int(self.time_span/self.time_interval),fargs=(ax,elems),interval=int(self.time_interval*1000),repeat=False)
            #self.ani.save("proto_type_no1.gif", writer = 'Pillow')
            plt.show()
    
    def one_step(self,time,ax,elems):
        print("[debug] World: one_step")
        while elems: elems.pop().remove()
        second="time= %.2f[s]"%(time*self.time_interval)
        elems.append(ax.text(-4.5,4.5,str(second),fontsize=10))
        for obj in self.objects:
            obj.draw(ax,elems)
            if hasattr(obj,"one_step"):obj.one_step(self.time_interval)

In [134]:
class IdealRobot:
    def __init__(self,pose,color="black",agent=None,sensor=None):
        self.pose=pose
        self.r=0.5
        self.color=color
        self.agent=agent
        self.sensor=sensor
        self.poses=[pose]
    
    def draw(self,ax,elems):
        print("[debug] IdealRobot: draw")
        x,y,theta=self.pose
        self.poses.append(self.pose)
        xn=x+self.r*math.cos(theta)
        yn=y+self.r*math.sin(theta)
        elems+=ax.plot([x,xn],[y,yn],color=self.color)
        c=patches.Circle(xy=(x,y),radius=self.r,color="black",fill=False)
        elems.append(ax.add_patch(c))
        diff=self.sensor.measurement(self.pose)
        
    
    @classmethod
    def state_transition(cls,nu,omega,time,pose):
        print("[debug] IdealRobot: state_transition")
        t0=pose[2]
        if math.fabs(omega)<1e-10:
            return pose+np.array([nu*math.cos(t0),
                                   nu*math.sin(t0),
                                   omega
                                   ])*time
        else:
            return pose+np.array([nu/omega*(math.sin(t0+omega*time)-math.sin(t0)),
                                   nu/omega*(-math.cos(t0+omega*time)+math.cos(t0)),
                                   omega*time])
        
    def one_step(self,time):
        print("[debug] IdealRobot: one_step")
        nu,omega=self.agent.decision()
        self.pose=self.state_transition(nu,omega,time,self.pose)

In [135]:
class Agent:
    def __init__(self,nu,omega):
        self.nu=nu
        self.omega=omega
    
    def decision(self,obs=None):
        print("[debug] Agent: decision")
        return self.nu,self.omega

In [136]:
class Map:
    def __init__(self):
        self.landmarks= []
    
    def append_landmark(self,lm):
        print("[debug] Map: append_landmark")
        lm.id=len(self.landmarks)
        self.landmarks.append(lm)
    
    def draw(self,ax,elems):
        print("[debug] Map: draw")
        for lm in self.landmarks:
            lm.draw(ax,elems)

In [137]:
class Landmark:
    def __init__(self,pos,id=None):
        self.pos=pos
        self.id=id
        
    def draw(self,ax,elems):
        print("[debug] Landmark: draw")
        elems.append(ax.scatter(self.pos[0],self.pos[1],s=100,marker="|",color="orange"))
        #elems.append(ax.text(self.pos[0],self.pos[1],"id:"+str(self.id),fontsize=10))

In [150]:
class IdealCamera:
    def __init__(self):
        self.map=None
        self.result=[]
    
    def capture(self,m):
        print("[debug] IdealCamera: capture")
        self.map=m
    
    def measurement(self,pose):
        print("[debug] IdealCamera: measurement")
        for lm in self.map.landmarks:
            xd=lm.pos[0]-pose[0]
            yd=lm.pos[1]-pose[1]
            distance=math.sqrt(xd**2+yd**2)
            dd=math.atan2(xd,yd)-pose[2]
            print("------------------------")
            print("------------------------")
            print("fai:%.1f, theta:%.1f"%(math.degrees(math.atan2(xd,yd)),math.degrees(pose[2])))
            print("distance:%.2f,  dd:%.1f"%(distance,math.degrees(dd)))
            while dd >= np.pi: dd -= 2*np.pi
            while dd < -np.pi: dd += 2*np.pi
            print("------------------------")
            print("distance:%.2f,  dd:%.1f"%(distance,math.degrees(dd)))
            set_rslt=(distance,dd)
            self.result.append(set_rslt)
        print(self.result)
        return self.result

In [151]:
world=World(15,0.1,debug=True)

m=Map()
v=0
for i in range(0,v,2): m.append_landmark(Landmark(np.array([-4,i*0.2-(v*0.2/2)])))
m.append_landmark(Landmark(np.array([3,3])))
world.append(m)
straight=Agent(0.5,0)
cm1=IdealCamera()
cm1.capture(m)
robot1=IdealRobot(np.array([0,0,math.pi]).T,agent=straight,sensor=cm1)
world.append(robot1)

world.draw()

[debug] Map: append_landmark
[debug] World: append
[debug] IdealCamera: capture
[debug] World: append
[debug] World: draw
[debug] World: one_step
[debug] Map: draw
[debug] Landmark: draw
[debug] IdealRobot: draw
[debug] IdealCamera: measurement
------------------------
------------------------
fai:45.0, theta:180.0
distance:4.24,  dd:-135.0
------------------------
distance:4.24,  dd:-135.0
[(4.242640687119285, -2.356194490192345)]
[debug] IdealRobot: one_step
[debug] Agent: decision
[debug] IdealRobot: state_transition
[debug] World: one_step
[debug] Map: draw
[debug] Landmark: draw
[debug] IdealRobot: draw
[debug] IdealCamera: measurement
------------------------
------------------------
fai:45.5, theta:180.0
distance:4.28,  dd:-134.5
------------------------
distance:4.28,  dd:-134.5
[(4.242640687119285, -2.356194490192345), (4.278142120126446, -2.3479302155326938)]
[debug] IdealRobot: one_step
[debug] Agent: decision
[debug] IdealRobot: state_transition
[debug] World: one_step
[deb